In [1]:
%%javascript
if(MathJax) {
    MathJax.Hub.Config({
        TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
    MathJax.Hub.Queue(
    ["resetEquationNumbers", MathJax.InputJax.TeX],
    ["PreProcess", MathJax.Hub],
    ["Reprocess", MathJax.Hub]
  );
}

<IPython.core.display.Javascript object>

## 誤差逆伝播法

計算グラフ - 計算の過程をグラフ構造で表したもの

計算グラフを誤差逆伝播法の説明に使う利点は，各過程での微分の計算を簡単かつ効率よく理解できること．

あるノードにおいて，

$$
x \longrightarrow f \longrightarrow y
$$

であり，$y$のLossが$E$のとき，

$$
E \frac{\partial y}{\partial x} \longleftarrow f \longleftarrow E
$$

というノードにおける局所的な微分を行い，それを次のノードに伝達していく．

なぜこの局所的な微分の伝達で全体の微分ができるかというと，それは以下の連鎖律（Chain Rule）によって説明できる

$$
\begin{equation}
z = (x + y)^2
\end{equation}
$$

は，

$$
\begin{equation}
z = t^2 \\
t = x + y
\end{equation}
$$

という2つの式の合成である．このとき，

> __連鎖律__ ある関数が合成関数で表される場合、その合成関数の微分は、合成関数を構成す るそれぞれの関数の微分の積によって表すことができる。

という性質を利用すると，$z$の$x$に関する微分は，

$$
\begin{equation}
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
\end{equation}
$$

と書くことができる．

$$
\frac{\partial t}{\partial x} = 1\\
\frac{\partial z}{\partial t} = 2t
$$

なので，結果として，

$$
\begin{equation}
\frac{\partial z}{\partial x} = 2(x+y)
\end{equation}
$$

となる


## 活性化関数の逆伝播

### ReLU

$$
\begin{equation}
y = \begin{cases}
x \quad (x > 0) \\
0 \quad (x \leqq 0)
\end{cases} \\

\frac{\partial y}{\partial x} = \begin{cases}
1 \quad (x > 0) \\
0 \quad (x \leqq 0)
\end{cases}
\end{equation}
$$

こちらは定義も実装も簡単である

### Sigmoid

シグモイド関数の微分は，答えは簡単なのだが，導出まで含めてきちんと見ておこう

$$
\begin{eqnarray}
y &=& \frac{1}{1 + exp(-x)} \\
&=& \frac{1}{1 + e^{-x}} \\
&=& (1 + e^{-x})^{-1} \\
\end{eqnarray}
$$

ここで，

$$
\begin{equation}
u = 1 + e^{-x} \\
f(x)=u^{-1}
\end{equation}
$$

とおくと，それぞれ微分して，

$$
\begin{eqnarray}
\frac{du}{dx} &=& -e^{-x} \\
\frac{dy}{du} &=& -u^{-2} \\
&=& -(1 + e^{-x})^{-2}
\end{eqnarray}
$$

ここに，

__合成関数の微分の公式__

$$
\begin{equation}
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
\end{equation}
$$

をあてはめる．

$$
\begin{eqnarray}
\frac{dy}{dx} &=& \frac{dy}{du} \cdot \frac{du}{dx} \\
&=& -(1 + e^{-x})^{-2} \cdot -e^{-x} \\
&=& (-1)^2 \cdot \frac{1}{(1 + e^{-x})^{2}} \cdot e^{-x} \\
&=& \frac{e^{-x}}{1 + e^{-x}} \cdot \frac{1}{1 + e^{-x}} \\
&=& \left( \frac{1 + e^{-x}}{1 + e^{-x}} - \frac{1}{1 + e^{-x}} \right) \cdot \frac{1}{1 + e^{-x}}
\end{eqnarray}
$$

ここで，そもそもシグモイド関数の定義が，$y=\frac{1}{1 + e^{-x}}$であったことから，

$$
\begin{equation}
\frac{dy}{dx} = (1 -y) \cdot y
\end{equation}
$$

となる．


In [4]:
import numpy as np
import sys, os
import os.path as path
sys.path.append(path.abspath(path.join(os.curdir ,"../..")))

from common.functions import sigmoid

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## Affineレイヤーの逆伝播

Affineとは，行列の内積のこと（幾何学分野でアフィン変換と呼ばれる）．`np.dot` ね．

![](imgs/affine.png)

![](imgs/affine1.png)

## 行列の和と積の誤差逆伝播法

証明: https://qiita.com/Matchlab/items/f327f1dcd143b52ae3bb

### 行列の和

$$
\boldsymbol{Y} = \boldsymbol{W} + \boldsymbol{X}
$$

のとき，

$$
\frac{\partial L}{\partial \boldsymbol X} = \frac{\partial L}{\partial \boldsymbol Z}
$$

$$
\frac{\partial L}{\partial \boldsymbol Y} = \frac{\partial L}{\partial \boldsymbol Z} \\
$$

### 行列の積

$$
\boldsymbol{Y} = \boldsymbol{W} \cdot \boldsymbol{X}
$$

のとき，

(式5.13)

$$
\frac{\partial L}{\partial \boldsymbol X} = \frac{\partial L}{\partial \boldsymbol Y} \cdot \boldsymbol W^T
$$

$$
\frac{\partial L}{\partial \boldsymbol W} = \boldsymbol X^T \frac{\partial L}{\partial \boldsymbol Y}
$$

TODO: これの証明を理解すること

> なぜ行列の形状に注意するかというと、行列の内積では、対応する次元の要素数を 一致させる必要があり、その一致を確認することで、式 (5.13) を導くことができる からです。たとえば、 ∂Y ∂L の形状が (3,)、W の形状が (2,3) であるとき、 ∂X ∂L の形状 が (2,) になるように、 ∂Y ∂L と W の内積を考えます（図5-26）。そうすれば、おのず と、式 (5.13) が導かれるというわけです

## Affineレイヤーの逆伝播（バッチ版）

![](imgs/affine2.png)

バッチ版の場合，$\boldsymbol B$のバイアスは，すべてのデータ列に対して加算されるので，逆伝播を計算するときには， 

$$
\frac{\partial L}{\partial \boldsymbol B} = \sum \frac{\partial L}{\partial \boldsymbol Y}_{axis=0}
$$

となる．

## Softmax with loss

![](imgs/softmax_simplified.png)

この図で注目すべきは、逆伝播の結果です。Softmax レイヤからの逆伝播は、 $(y_1 − t_1, y_2 − t_2, y_3 − t_3)$ という“キレイ”な結果になっています。 $(y_1, y_2, y_3)$ はSoftmax レイヤの出力、$(t_1, t_2, t_3)$ は教師データなので、$(y_1 − t_1, y_2 − t_2, y_3 − t_3)$は、Softmax レイヤの出力と教師ラベルの差分になります。ニューラルネットワークの逆伝播では、この差分である誤差が前レイヤへ伝わっていくのです。これはニュー ラルネットワークの学習における重要な性質です。

実は、そのような“キレイ”な結果は偶然ではなく、そうなるように交差エントロピー誤差という関数が設計されたのです


In [14]:
from common.functions import sigmoid, cross_entropy_error

class Affine:
    def __init__(self, W, b):
        self.params = {}
        self.W = W
        self.b = b
        self.dW = None
        self.db = None
        self.x = None

    def forward(self, x):
        # テンソル対応
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out
    
    # dout = dL/dY 
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None # softmaxの出力
        self.t = None # 教師データ

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 教師データがone-hot-vectorの場合
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx
    
from common.functions import sigmoid, softmax, cross_entropy_error
from common.gradient import numerical_gradient
from collections import OrderedDict

class Tom2Net:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        self.layers = OrderedDict() 
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu() 
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for l in self.layers.values():
            x = l.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accu = np.sum(y == t) / float(x.shape[0])
        return accu
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # forward 
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for l in layers:
            dout = l.backward(dout)
        
        grads = {} 
        grads['W1'] = self.layers['Affine1'].dW 
        grads['b1'] = self.layers['Affine1'].db 
        grads['W2'] = self.layers['Affine2'].dW 
        grads['b2'] = self.layers['Affine2'].db
        return grads

In [15]:
# Tom2Netを使った学習

from dataset.mnist import load_mnist

# データの読み込み
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = Tom2Net(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 勾配
    grad = network.gradient(x_batch, t_batch)
    
    # 更新
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.1009 0.1011
0.9052166666666667 0.9059
0.9233166666666667 0.9265
0.9364666666666667 0.9369
0.94555 0.9438
0.9507 0.9467
0.9557666666666667 0.9528
0.9603333333333334 0.9579
0.9650166666666666 0.9596
0.9685 0.9618
0.9699666666666666 0.9631
0.9704833333333334 0.9639
0.9734166666666667 0.9658
0.9741666666666666 0.9654
0.9758333333333333 0.9682
0.97735 0.968
0.9789666666666667 0.9687
